In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import preprocessing

In [175]:
df = pd.read_csv("Original Datasets/compas-scores-two-years.csv",sep=",")

In [176]:
df.head()

,age,c_charge_degree,race,sex,priors_count,decile_score
0,69,F,Other,Male,0,1
1,34,F,African-American,Male,0,3
2,24,F,African-American,Male,4,4
3,44,M,Other,Male,0,1
4,41,F,Caucasian,Male,14,6


In [215]:
dfNum = df.select_dtypes(include=['integer'])

In [178]:
def isAble(df,decisionCol):
    error = ""
    if df.isna().values.any() == True or df.isnull().values.any() == True:
        error = "Dataframe contains Null or NaN values"
    if df.shape[0] < 2 or df.shape[1] < 2:
        error ="Dataframe has less than 2 columns or less than 2 rows"
    if decisionCol not in df.columns:
        error = "Dataframe has not the decision column "+decisionCol
    return error

In [179]:
isAble(df,"decile_score")

''

In [180]:
df.describe()

,age,priors_count,decile_score
count,6172.000000,6172.000000,6172.000000
mean,34.534511,3.246436,4.418503
std,11.730938,4.743770,2.839463
min,18.000000,0.000000,1.000000
25%,25.000000,0.000000,2.000000
50%,31.000000,1.000000,4.000000
75%,42.000000,4.000000,7.000000
max,96.000000,38.000000,10.000000


In [181]:
categorical = df.select_dtypes(include=['object']).columns.tolist()

In [182]:
numerical = [i for i in df if i not in categorical]

In [183]:
numerical

['age', 'priors_count', 'decile_score']

In [186]:
def getQuartiles(column):
    if column in df.columns:
        p25 = df[column].quantile(0.25)
        median = df[column].quantile(0.5)
        p75 = df[column].quantile(0.75)
        return p25,median,p75
    return None
    

In [187]:
getQuartiles('age')

(25.0, 31.0, 42.0)

In [216]:
dfNum.head()

,age,priors_count,decile_score
0,69,0,1
1,34,0,3
2,24,4,4
3,44,0,1
4,41,14,6


In [217]:
def categorize4(row,column):

    p25,median,p75 = getQuartiles(column)
    if p25 == 0:
        colName = 'equal_zero'
    else: colName = 'lower_than_'+str(p25)
    if row <= p25: return colName
    elif row > p25 and row <= median: return 'between_'+('%f' % p25).rstrip('.0')+'_and_'+('%f' % median).rstrip('.0')
    elif row > median and row <= p75: return 'between_'+('%f' % median).rstrip('.0')+'_and_'+('%f' % p75).rstrip('.0')
    else: return 'bigger_than_'+('%f' % p75).rstrip('.0')

In [218]:
def categorize(df,numerical):
    for i in numerical:
        print(i)
        dfNum[i]=dfNum.apply(lambda x: categorize4(x[i],i),axis=1)

In [219]:
categorize(df,numerical)

age


C:\Users\blair\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


priors_count
decile_score


In [220]:
dfNum.head()

,age,priors_count,decile_score
0,bigger_than_42,equal_zero,lower_than_2.0
1,between_31_and_42,equal_zero,between_2_and_4
2,lower_than_25.0,between_1_and_4,between_2_and_4
3,bigger_than_42,equal_zero,lower_than_2.0
4,between_31_and_42,bigger_than_4,between_4_and_7


In [41]:
p25 = [i for i in df.age.values if i > df.age.quantile(0.25)]

In [24]:
def adapt(df):
    categorical = df.select_dtypes(include=['object']).columns.tolist()
    numerical = [i for i in df if i not in categorical]
    

['age', 'age_cat', 'priors_count', 'decile_score']

In [ ]:
def fixNumerical(df,numerical):
    for i in numerical:
        

In [58]:
df = df.dropna()

In [5]:
df.shape

(1000, 5)

In [9]:
df.head()

,Age_Young,Age_Older,Gender,Expenditures_Low,Expenditures_High,Ethnicity
0,1,0,Female,1,0,White not Hispanic
1,0,1,Male,0,1,White not Hispanic
2,1,0,Male,1,0,Hispanic
3,0,1,Female,1,0,Hispanic
4,1,0,Male,1,0,White not Hispanic


In [34]:
df = df.drop(["Unnamed: 0","Employee Name"],axis=1)

In [76]:
df["sex"].value_counts()

male      676
female    662
Name: sex, dtype: int64

In [7]:
df.describe()

,Age,Expenditures
count,1000.000000,1000.000000
mean,22.800000,18065.786000
std,18.462038,19542.830884
min,0.000000,222.000000
25%,12.000000,2898.750000
50%,18.000000,7026.000000
75%,26.000000,37712.750000
max,95.000000,75098.000000


In [51]:
import time

In [52]:
time.strftime('%H:%M:%S', time.gmtime(60))

'00:01:00'

In [10]:
categorical = df.select_dtypes(include=['object']).columns.tolist()
categorical

['Gender', 'Ethnicity']

In [11]:
#Deep copy the original data
data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)

Feature Gender
mapping {'Male': 1, 'Female': 0}
Feature Ethnicity
mapping {'Hispanic': 3, 'Asian': 1, 'Multi Race': 4, 'White not Hispanic': 7, 'Native Hawaiian': 5, 'American Indian': 0, 'Black': 2, 'Other': 6}


In [12]:
# Create new dataframe with dummy features
categorical_feats = df.select_dtypes(include=['object']).columns.tolist()
print(categorical_feats)

['Gender', 'Ethnicity']


In [13]:
df = pd.get_dummies(df, columns= categorical_feats)

In [14]:
df.head()

,Age_Young,Age_Older,Expenditures_Low,Expenditures_High,Gender_Female,Gender_Male,Ethnicity_American Indian,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Hispanic,Ethnicity_Multi Race,Ethnicity_Native Hawaiian,Ethnicity_Other,Ethnicity_White not Hispanic
0,1,0,1,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,1,0,1,0,0,0,0,0,0,0,1
2,1,0,1,0,0,1,0,0,0,1,0,0,0,0
3,0,1,1,0,1,0,0,0,0,1,0,0,0,0
4,1,0,1,0,0,1,0,0,0,0,0,0,0,1


In [15]:
df.to_csv("DDSData Binary.csv",index=None,sep=";")